In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

df_data = pd.read_pickle('data/small_set.xz')

In [15]:

#Keep measurement locations that were active by removing NaNs
#And sort to group measurement locations and order by timestamp
data = df_data[np.isfinite(df_data['duration'])].sort_values(by=['id','timestamp'])

#Linear interpolate for durations that are -1
# fp = duration that are not -1
# xp = position of those durations in the list
# positions in the range that are not in xp will get interpolated by the function np.interp

fp = data[(data.duration > 0)].duration.values
xp = [i for i,x in enumerate(data.duration) if x > 0]
inter = np.interp(range(0,data.duration.size),np.transpose(xp),fp)

pd.options.mode.chained_assignment = None  # default='warn'
data['duration_interpolation'] = inter

#Show the interpolation for one of the measurement locations
#X = range(data[(data.id == 'RWS01_MONIBAS_0581hrr0137ra0')].duration.size)
#plt.figure(figsize=(18,10))
#plt.plot(X, data[(data.id == 'RWS01_MONIBAS_0581hrr0137ra0')].duration)
#plt.plot(X, data[(data.id == 'RWS01_MONIBAS_0581hrr0137ra0')].duration_interpolation)
#plt.show()

#calculate the median of the durations to get the duration during normal hours
df_median = data.groupby(['id']).median()[['duration_interpolation']]
df_median.columns = ['duration_interpolation_median']
df = pd.merge(data, df_median, right_index=True, left_on='id')

#calculate the deviation compared to the median
df['duration_deviation'] = df.duration_interpolation - df.duration_interpolation_median

#EXAMPLE select a subset of the locations which are in a row
df_filtered = df[df['id'].isin(['RWS01_MONIBAS_0581hrr0192ra0','RWS01_MONIBAS_0581hrr0195ra0','RWS01_MONIBAS_0581hrr0207ra0','RWS01_MONIBAS_0581hrr0215ra0','RWS01_MONIBAS_0581hrr0221ra0','RWS01_MONIBAS_0581hrr0226ra0','RWS01_MONIBAS_0581hrr0233ra0','RWS01_MONIBAS_0581hrr0239ra0','RWS01_MONIBAS_0581hrr0244ra0','RWS01_MONIBAS_0581hrr0251ra0','RWS01_MONIBAS_0581hrr0257ra0','RWS01_MONIBAS_0581hrr0263ra0','RWS01_MONIBAS_0581hrr0269ra0','RWS01_MONIBAS_0581hrr0275ra0','RWS01_MONIBAS_0581hrr0281ra0','RWS01_MONIBAS_0581hrr0287ra0','RWS01_MONIBAS_0581hrr0293ra0','RWS01_MONIBAS_0581hrr0298ra0','RWS01_MONIBAS_0581hrr0309ra0','RWS01_MONIBAS_0581hrr0315ra0','RWS01_MONIBAS_0581hrr0320ra0','RWS01_MONIBAS_0581hrr0326ra0','RWS01_MONIBAS_0581hrr0332ra0','RWS01_MONIBAS_0581hrr0337ra0','RWS01_MONIBAS_0581hrr0342ra0','RWS01_MONIBAS_0581hrr0346ra0','RWS01_MONIBAS_0581hrr0352ra0','RWS01_MONIBAS_0581hrr0356ra0','RWS01_MONIBAS_0581hrr0361ra0','RWS01_MONIBAS_0581hrr0364ra0','RWS01_MONIBAS_0581hrr0369ra0','RWS01_MONIBAS_0581hrr0376ra0','RWS01_MONIBAS_0581hrr0383ra0','RWS01_MONIBAS_0581hrr0388ra0','RWS01_MONIBAS_0581hrr0394ra0'])]

#Or, select all locations
#df_filtered = df

df_pivot = df_filtered.pivot(index='timestamp', columns='id', values='duration_deviation').clip(-60, 60)

# fig, ax = plt.subplots(figsize=(25,100)) 
# ax = sns.heatmap(df_pivot, center=0, xticklabels=False, yticklabels=False, cbar=False, cmap="YlGnBu")

#sns.plt.show()

In [16]:
df_filtered.iloc[0]

id                               RWS01_MONIBAS_0581hrr0192ra0
latitude                                              51.4932
length                                                    290
longitude                                             5.31415
names                                          0581hrr0192ra0
duration                                                    9
timestamp                                2018-06-07T06:44:00Z
duration_interpolation                                      9
duration_interpolation_median                              10
duration_deviation                                         -1
Name: 42, dtype: object

In [17]:
snap_time = df_filtered.timestamp.iloc[0]
snapshot = df_filtered[df_filtered.timestamp==snap_time]

In [18]:
snapshot[:10]

,id,latitude,length,longitude,names,duration,timestamp,duration_interpolation,duration_interpolation_median,duration_deviation
42,RWS01_MONIBAS_0581hrr0192ra0,51.49323,290,5.31415,0581hrr0192ra0,9.0,2018-06-07T06:44:00Z,9.0,10.0,-1.0
44,RWS01_MONIBAS_0581hrr0195ra0,51.49379,1260,5.31008,0581hrr0195ra0,41.0,2018-06-07T06:44:00Z,41.0,43.0,-2.0
47,RWS01_MONIBAS_0581hrr0207ra0,51.50074,765,5.29611,0581hrr0207ra0,25.0,2018-06-07T06:44:00Z,25.0,26.0,-1.0
49,RWS01_MONIBAS_0581hrr0215ra0,51.50455,550,5.28703,0581hrr0215ra0,18.0,2018-06-07T06:44:00Z,18.0,19.0,-1.0
51,RWS01_MONIBAS_0581hrr0221ra0,51.50663,530,5.27984,0581hrr0221ra0,18.0,2018-06-07T06:44:00Z,18.0,18.0,0.0
53,RWS01_MONIBAS_0581hrr0226ra0,51.50865,670,5.27292,0581hrr0226ra0,23.0,2018-06-07T06:44:00Z,23.0,23.0,0.0
55,RWS01_MONIBAS_0581hrr0233ra0,51.51120,660,5.26418,0581hrr0233ra0,23.0,2018-06-07T06:44:00Z,23.0,23.0,0.0
57,RWS01_MONIBAS_0581hrr0239ra0,51.51385,480,5.25566,0581hrr0239ra0,17.0,2018-06-07T06:44:00Z,17.0,16.0,1.0
59,RWS01_MONIBAS_0581hrr0244ra0,51.51577,710,5.24947,0581hrr0244ra0,25.0,2018-06-07T06:44:00Z,25.0,24.0,1.0
61,RWS01_MONIBAS_0581hrr0251ra0,51.51859,600,5.24029,0581hrr0251ra0,21.0,2018-06-07T06:44:00Z,21.0,21.0,0.0


In [19]:
from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, Range1d, PanTool, WheelZoomTool, BoxSelectTool
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
# from bokeh.palettes import Viridis5
try:
    from api import API_KEY
except ImportError:
    raise FileNotFoundError("No API key found. Create one at https://developers.google.com/maps/documentation/javascript/get-api-key and save the line `API_KEY='your key'` in api.py")
    
%matplotlib
map_options = GMapOptions(lat=52.0687206, lng=4.9863232, map_type="roadmap", zoom=7)
plot = GMapPlot(x_range=Range1d(),y_range=Range1d(),map_options=map_options)
plot.title.text = "Test plot"
plot.api_key = API_KEY

source=ColumnDataSource(data=dict(lat=snapshot.latitude.tolist(),lon=snapshot.longitude.tolist(),size=snapshot.duration_deviation.values))
circle=Circle(x="lon",y="lat",size="size")
plot.add_glyph(source,circle)
plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
show(plot)



Using matplotlib backend: TkAgg
